<a href="https://colab.research.google.com/github/SnrPep/Mos/blob/main/lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import random
import math

In [7]:
#Ядро
class Value:
    def __init__(self, data, _children=(), _op=''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')
        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward
        return out

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out

    def __sub__(self, other):
        return self + (other * -1)

    def __neg__(self):
        return self * -1

    def __abs__(self):
        out = Value(abs(self.data), (self,), 'abs')
        def _backward():
            self.grad += (1.0 if self.data >= 0 else -1.0) * out.grad
        out._backward = _backward
        return out

    def tanh(self):
        t = (math.exp(2 * self.data) - 1) / (math.exp(2 * self.data) + 1)
        out = Value(t, (self,), 'tanh')
        def _backward():
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward
        return out

    def backward(self):
        topo = []
        visited = set()
        def build(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build(child)
                topo.append(v)
        build(self)
        self.grad = 1.0
        for node in reversed(topo):
            node._backward()

#Нейрон
class Neuron:
    def __init__(self, nin):
        self.w = [Value(random.uniform(-1, 1)) for _ in range(nin)]
        self.b = Value(0.0)

    def __call__(self, x):
        act = sum((wi * xi for wi, xi in zip(self.w, x)), self.b)
        return act.tanh()

#Слой
class Layer:
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]

    def __call__(self, x):
        return [neuron(x) for neuron in self.neurons]

#Модель
class NeuralNet:
    def __init__(self, sizes):
        self.layers = []
        for i in range(len(sizes) - 1):
            self.layers.append(Layer(sizes[i], sizes[i+1]))

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def parameters(self):
        params = []
        for layer in self.layers:
            for neuron in layer.neurons:
                params.extend(neuron.w + [neuron.b])
        return params

#Функция потерь — средняя абсолютная ошибка
def mean_absolute_error(preds, targets):
    losses = [abs(pred - Value(y)) for pred, y in zip(preds, targets)]
    return sum(losses, Value(0.0)) * (1 / len(losses))


In [11]:
#Датасет
df = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv')
X = df[['sepal_length', 'sepal_width', 'petal_length']].values.tolist()
y = df['petal_width'].tolist()

#Модель и обучение
net = NeuralNet([3, 6, 4, 1])
epochs = 300
lr = 0.05

for epoch in range(epochs):
    total_loss = 0.0
    for xi, yi in zip(X, y):
        x_val = [Value(val) for val in xi]
        out = net(x_val)
        loss = mean_absolute_error(out, [yi])

        for p in net.parameters():
            p.grad = 0.0
        loss.backward()

        for p in net.parameters():
            p.data -= lr * p.grad

        total_loss += loss.data

    if epoch % 50 == 0:
        print(f"Эпоха {epoch}: Потеря = {total_loss/len(X):.4f}")


Эпоха 0: Потеря = 0.5420
Эпоха 50: Потеря = 0.4910
Эпоха 100: Потеря = 0.4844
Эпоха 150: Потеря = 0.4870
Эпоха 200: Потеря = 0.4843
Эпоха 250: Потеря = 0.4842


In [12]:
#Проверка
print("\nПримеры предсказаний:")
for i in range(5):
    inp = [Value(v) for v in X[i]]
    pred = net(inp)[0].data
    print(f"Настоящее: {y[i]:.2f}, Предсказание: {pred:.2f}")


Примеры предсказаний:
Настоящее: 0.20, Предсказание: 0.12
Настоящее: 0.20, Предсказание: 0.12
Настоящее: 0.20, Предсказание: 0.12
Настоящее: 0.20, Предсказание: 0.12
Настоящее: 0.20, Предсказание: 0.12
